Basic chatbot to act as a clinician to diagnose clinical cases

Load required libraries

see the following URL
https://docs.science.ai.cam.ac.uk/hands-on-llms/2_open_ai/



In [1]:
from openai import OpenAI

In [2]:
import os
import dotenv
from rich import print as rprint

Call chatbot OpenAI API

In [3]:
dotenv.load_dotenv()
# load .env

# get API KEY and store it in variable
OPENAI_API_KEY = os.getenv('OPENAI_API_KEY')

# call OpenAI function
client = OpenAI()


Call API

In [4]:
system_prompt = "You are a clinician and you are diagnosing cases. Given the prompt please give a diagnosis. Please explain all the steps in your reasoning process."
user_query = "A patient has come to you complaining about chest pain. Please give a diagnosis. Please explain all the steps."

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_query}
    ],
    max_tokens=1000 #,
    #temperature=2
)

print(response.choices[0].message.content)


To diagnose a patient presenting with chest pain, I would need to follow a systematic approach, encompassing history taking, physical examination, differential diagnosis, and potentially diagnostic testing. Here’s how I would approach it:

### Step 1: History Taking
1. **Symptom Characteristics**: I would ask the patient to describe the chest pain in detail:
   - **Location**: Where is the pain located? (e.g., left side, center)
   - **Quality**: How does it feel? (e.g., sharp, dull, burning, squeezing)
   - **Intensity**: On a scale of 1 to 10, how severe is the pain?
   - **Duration**: How long does it last? Is it constant or intermittent?
   - **Triggers/Relieving Factors**: What seems to trigger it? Does anything relieve it?
   - **Associated Symptoms**: Are there other symptoms present? (e.g., shortness of breath, nausea, sweating, palpitations)

2. **Medical History**: I would inquire about any relevant medical history, such as:
   - Previous episodes of chest pain
   - Cardiovas

Slightly improved version of chatbot with more clinical case information

In [5]:
# read in file with case histories
f_str = open("master_prompt.txt", "r")
master_prompt = f_str.read()

system_prompt = "You are a clinician and you are diagnosing cases. Given the prompt please give a diagnosis. Please explain all the steps in your reasoning process."
user_query = "A patient has come to you complaining about chest pain. Please give a diagnosis. Please explain all the steps."
user_query = user_query + master_prompt

#print(user_query)

response = client.chat.completions.create(
    model="gpt-4o-mini",
    messages=[
        {"role":"system", "content":system_prompt},
        {"role":"user", "content":user_query}
    ],
    max_tokens=1000 #,
    #temperature=2
)

print("*************************************************\n")
print("Output of ClinicalAssistant: \n")
print(response.choices[0].message.content)
print("*************************************************\n")



*************************************************

Output of ClinicalAssistant: 

Based on the clinical case and information provided, the diagnosis for this patient is **viral pericarditis**.

**Steps in the reasoning process:**

1. **Chief Complaint and Symptom Analysis**:
   - The patient presents with acute chest pain that is central, radiating to both sides of the chest, which began 4 hours prior to the visit. The persistent nature and central location of the pain lend it to be concerning for a cardiac etiology. 
   - The pain is relieved by sitting up and leaning forward, which is characteristic of pericarditis rather than myocardial infarction or other causes of chest pain.
   - There is also a history of occasional, brief, left-sided chest pain for the past two years, but these previous episodes seem less significant, as they did not last long or exhibit concerning features.

2. **Review of Associated Symptoms**:
   - The patient does not report symptoms commonly associated wit

Create a chatbot with memory of previous conversation history

In [ ]:
print("************************************\n")
print("Intelligent clinical chatbot with memory: \n")

# create a data structure list/dict
system_state = {
    "conversation_history":[]
    }

# create a systemp prompt
#system_prompt = system_prompt + master_prompt

system_state["conversation_history"].append({
     "role":"system",
    "content": system_prompt
 })


while True:
    user_input = input("User: ")

    if user_input.lower() in ['exit','done', 'endchat']:
        print("Assistant: Bye! \n")
        break

    print(user_input)    

    # append user input to system_state
    system_state["conversation_history"].append({ "role":"user", "content":user_input })

    # call OpenAI API
    response_openai = client.chat.completions.create(
        model="gpt-4o-mini",
        messages=system_state["conversation_history"],
        max_tokens=1000 #,
        # temperature = 1.0
    )

    # save the output of the OpenAI chatbot to system_state
    system_state["conversation_history"].append(
        {
            "role":"assistant",
            "content":response_openai.choices[0].message.content
        }
    )  

    # print output of chatbot in this iteration
    print("**************************************************\n")
    print("Output of chatbot ...... \n")
    print(response_openai.choices[0].message.content)
    print("End output \n")
    print("**************************************************\n")


************************************

Intelligent clinical chatbot with memory: 

hello
**************************************************

Output of chatbot ...... 

Hello! How can I assist you today? If you have a case or a question in mind, please provide the details, and I'll do my best to help you with a diagnosis or guidance.
End output 

**************************************************

you see a patient with a viral infection and he has chest pain. suggest a diagnosis
**************************************************

Output of chatbot ...... 

When considering a patient with a viral infection who presents with chest pain, I would follow a systematic approach to arrive at a differential diagnosis. Here are the steps in my reasoning process:

1. **History Taking**: I would gather detailed information about the patient's symptoms, including:
   - Duration and character of the chest pain (sharp, dull, pleuritic, etc.)
   - Any associated symptoms (cough, shortness of breath, fe